# Experimenting SHAP with multivariate sequential data, on PyTorch
---

Although SHAP works pretty great for most use cases, providing realistic and well founded interpretability to machine learning models, it doesn't seem to be able to handle multivariate sequential data. The issue appears that it doesn't successfully filter out the padding values (which are needed to form tensors when the data has variable sequence length).

This notebook serves as a simple example to illustrate and debug SHAP for this multivariate sequential data scenarios, using the PyTorch framework.

## Importing the necessary packages

In [ ]:
import pandas as pd              # Pandas to handle the data in dataframes
import numpy as np               # Math operations with NumPy
import torch                     # PyTorch to create and apply deep learning models
import shap                      # Model-agnostic interpretability package inspired on Shapley values
import utils                     # Contains auxiliary functions
from Time_Series_Dataset import Time_Series_Dataset # Dataset subclass which allows the creation of Dataset objects
from NeuralNetwork import NeuralNetwork # Import the neural network model class

In [ ]:
# Debugging packages
import pixiedust                 # Debugging in Jupyter Notebook cells
import time                      # Calculate code execution time

## Creating a dummy dataset

In [ ]:
dmy_data = np.array([[0, 0, 23, 284, 70, 5, 0],
                     [0, 1, 24, 270, 73, 5, 0],
                     [0, 2, 22, 290, 71, 5, 0],
                     [0, 3, 20, 288, 65, 4, 1],
                     [0, 4, 21, 297, 64, 4, 1],
                     [1, 0, 25, 300, 76, 5, 0],
                     [1, 1, 19, 283, 70, 5, 0],
                     [1, 2, 19, 306, 59, 5, 1],
                     [1, 3, 18, 298, 55, 3, 1],
                     [2, 0, 20, 250, 70, 5, 0],
                     [2, 1, 20, 254, 68, 4, 1],
                     [2, 2, 19, 244, 70, 3, 1],
                     [3, 0, 27, 264, 78, 4, 0],
                     [3, 1, 22, 293, 67, 4, 1]])

In [ ]:
dmy_df = pd.DataFrame(dmy_data, columns=['subject_id', 'ts', 'Var0', 'Var1', 'Var2', 'Var3', 'label'])
dmy_df

In [ ]:
# List of used features
dmy_cols = list(dmy_df.columns)

# Remove features that aren't used by the model to predict the label
for unused_feature in ['subject_id', 'ts', 'label']:
    dmy_cols.remove(unused_feature)

In [ ]:
dmy_cols

## Setting up model parameters and constants

### Neural network and dataset parameters

In [ ]:
n_patients = dmy_df.subject_id.nunique()     # Total number of patients
n_inputs = len(dmy_df.columns)               # Number of input features
n_hidden = 2                                 # Number of hidden units
n_outputs = 1                                # Number of outputs
n_layers = 1                                 # Number of LSTM layers
p_dropout = 0.2                              # Probability of dropout

### Training parameters

In [ ]:
batch_size = 3                                  # Number of patients in a mini batch
n_epochs = 50                                   # Number of epochs
lr = 0.001                                      # Learning rate

### Sequence length dictionary

(number of temporal events) of each sequence (patient)

In [ ]:
seq_len_df = dmy_df.groupby('subject_id').ts.count().to_frame().sort_values(by='ts', ascending=False)
seq_len_dict = dict([(idx, val[0]) for idx, val in list(zip(seq_len_df.index, seq_len_df.values))])

In [ ]:
seq_len_dict

## Preparing the dataset

### Normalizing the features

In [ ]:
dmy_norm_df = utils.normalize_data(dmy_df, see_progress=False)
dmy_norm_df

### Padding

Pad the data so that all sequences have the same length (so that it can be converted to a PyTorch tensor).

In [ ]:
padding_value = 999999

In [ ]:
data = utils.dataframe_to_padded_tensor(dmy_norm_df, seq_len_dict, n_patients, n_inputs, padding_value=padding_value)

In [ ]:
data

### Dataset object

In [ ]:
dataset = Time_Series_Dataset(data, dmy_norm_df)

### Separating into train and validation sets

Since this notebook is only for SHAP debugging purposes, with a very small dummy dataset, we'll not be using a test set.

In [ ]:
# Get the train and validation sets data loaders, which will allow loading batches
train_dataloader, val_dataloader = utils.create_train_sets(dataset, validation_ratio=0.25, batch_size=batch_size, 
                                                           get_indeces=False)

In [ ]:
next(iter(train_dataloader))[0]

In [ ]:
next(iter(val_dataloader))[0]

## Training the model

### Initialize model

In [ ]:
# Instantiate the model (removing the two identifier columns and the labels from the input size)
model = NeuralNetwork(n_inputs-3, n_hidden, n_outputs, n_layers, p_dropout)

### Running the training process

In [ ]:
model = utils.train(model, train_dataloader, val_dataloader, seq_len_dict, batch_size, n_epochs, 
                    lr, model_path='models/', padding_value=padding_value)

## Interpretability / SHAP

In [ ]:
# Make sure the model is in evaluation mode, with dropout turned off
model.eval()

In [ ]:
features, labels = dataset.X, dataset.y

In [ ]:
features, labels, x_lengths = utils.sort_by_seq_len(features, seq_len_dict, labels=labels)

Get an overview of the model's output for each sample:

In [ ]:
ref_output = model(features[:, :, 2:].float(), x_lengths)

In [ ]:
ref_output

In [ ]:
# Remove indeces that correspond to outputs in padded data
real_idx = [idx for n_subject in range(features.shape[0])
            for idx in range(n_subject*features.shape[1], n_subject*features.shape[1]+x_lengths[n_subject])]
real_idx

In [ ]:
# Indeces at the end of each sequence
final_seq_idx = [n_subject*features.shape[1]+x_lengths[n_subject]-1 for n_subject in range(features.shape[0])]
final_seq_idx

In [ ]:
# Ignore outputs from padded data
ref_output = ref_output[real_idx]

In [ ]:
ref_output_s = pd.Series([float(x) for x in list(ref_output.detach().numpy())])

In [ ]:
# Get an overview of the important features and model output for the current patient
dmy_df.assign(output=ref_output_s)

### Kernel Explainer

In [ ]:
# Function that will be used in the kernel explainer, converting a dataframe object into the model's output
# def f(data, col_names, hidden_state=None, seq_id_col='subject_id', inst_id_col='ts', label_col='label'):
def f(data, hidden_state=None):
#     # Convert the data into a dataframe format
#     df = pd.DataFrame(data, columns=col_names)
    
#     # Get the original length of each sequence of the dataframe
#     # [TODO] Consider saving this sequence length dictionary in the model object, to avoid an unefficient repeated process
#     seq_len_df = df.groupby(seq_id_col)[inst_id_col].count().to_frame().sort_values(by=inst_id_col, ascending=False)
#     seq_len_dict = dict([(idx, val[0]) for idx, val in list(zip(seq_len_df.index, seq_len_df.values))])
    
#     # Dataset parameters
#     n_subjects = df.subject_id.nunique()     # Total number of subjects
#     n_inputs = len(df.columns)               # Number of input features
    
#     # Pad and convert the dataframe into a PyTorch tensor
#     padding_value = 999999
#     data = utils.dataframe_to_padded_tensor(df, seq_len_dict, n_subjects, n_inputs, padding_value=padding_value)
    
#     # Find and remove the indeces of the columns that can't be fed to the model
#     label_idx = [idx for idx, col in enumerate(df.columns) if col is label_col][0]
#     features_idx = list(range(data.shape[2]))
#     features_idx.remove(label_idx)
#     features = data[:, :, features_idx]
    
#     # Separate the labels from the features
#     labels = data[:, :, label_idx]
#     features_idx = list(range(data.shape[2]))
#     features_idx.remove(label_idx)
#     features = data[:, :, features_idx]

#     # Get the original sequence lengths and sort the data by them
#     features, labels, x_lengths = utils.sort_by_seq_len(features, seq_len_dict, labels=labels)
    
    # Make sure the data is of type float
#     features = features.float()
#     labels = labels.float()
    data = torch.from_numpy(data).unsqueeze(0).float()
    
    # Calculate the output
#     output = model(features[:, :, 2:], x_lengths, hidden_state=hidden_state)
    output = model(data, hidden_state=hidden_state)
    
#     # Indeces at the end of each sequence
#     final_seq_idx = [n_subject*features.shape[1]+x_lengths[n_subject]-1 for n_subject in range(features.shape[0])]
#     # Remove indeces that correspond to outputs in padded data
#     real_idx = [idx for n_subject in range(features.shape[0])
#                 for idx in range(n_subject*features.shape[1], n_subject*features.shape[1]+x_lengths[n_subject])]

#     # Get the outputs of the last instances of each sequence
#     output = output[real_idx]
    return output.detach().numpy()

In [ ]:
# outputs = f(dmy_norm_df.values, dmy_norm_df.columns)
outputs = f(dmy_norm_df.values[:, 2:-1])
outputs

In [ ]:
# %%pixie_debugger
explainer = shap.KernelExplainer(f, dmy_norm_df.values, kwargs={'keep_index': False, 'isRNN': True})

In [ ]:
# %%pixie_debugger
shap_values = explainer.shap_values(dmy_norm_df, kwargs={'isRNN': True, 'model_obj': model, 'l1_reg': 'aic'})

In [ ]:
shap_values

In [ ]:
shap_values.reshape(-1, model.rnn.input_size).shape

In [ ]:
features[:, :, 2:].contiguous().view(-1, model.rnn.input_size).numpy().shape

In [ ]:
# Summarize the effects of all the features
shap.summary_plot(shap_values.reshape(-1, model.rnn.input_size), features=features[:, :, 2:].contiguous().view(-1, model.rnn.input_size).numpy(), feature_names=dmy_cols, plot_type='violin')

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
subject = 0

# True sequence length of the current patient's data
seq_len = seq_len_dict[features[subject, 0, 0].item()]

# Plot the explanation of the predictions for one subject
shap.force_plot(explainer.expected_value[0], shap_values[subject, :seq_len], features=features[subject, :seq_len, 2:].numpy(), feature_names=dmy_cols)

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
ts = 3

# Plot the explanation of the predictions for one subject
shap.force_plot(explainer.expected_value[0], shap_values[subject, ts], features=features[subject, ts, 2:].numpy(), feature_names=dmy_cols)

### Deep Explainer

Using the standard SHAP library, as with the line of code bellow, doesn't work well as the model needs to know the true sequence length (as an argument of its feedforward method). Otherwise, it assumes that the paddings are real values.

In [ ]:
explainer = shap.DeepExplainer(model, features[:, :, 2:].float())

In [ ]:
shap_values = explainer.shap_values(features[:, :, 2:].float())

In [ ]:
# Summarize the effects of all the features
shap.summary_plot(shap_values.reshape(-1, model.rnn.input_size), features=features[:, :, 2:].contiguous().view(-1, model.rnn.input_size).numpy(), feature_names=dmy_cols, plot_type='violin')

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
subject = 1

# True sequence length of the current patient's data
seq_len = seq_len_dict[features[subject, 0, 0].item()]

# Plot the explanation of the predictions for one subject
shap.force_plot(explainer.expected_value[0], shap_values[subject, :seq_len], features=features[subject, :seq_len, 2:].numpy(), feature_names=dmy_cols)

In [ ]:
ref_output = model(features[subject, :, 2:].float().unsqueeze(0), [x_lengths[subject]])

In [ ]:
ref_output_s = pd.Series([float(x) for x in list(ref_output.detach().numpy())])

In [ ]:
# Get an overview of the important features and model output for the current patient
dmy_df[dmy_df.subject_id == subject].reset_index().drop(columns='index').assign(output=ref_output_s)

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
ts = 3

# Plot the explanation of the predictions for one subject
shap.force_plot(explainer.expected_value[0], shap_values[subject, ts], features=features[subject, ts, 2:].numpy(), feature_names=dmy_cols)

Using the adapted version of the SHAP library, as with the line of code bellow, makes the relative feature importance make sense ("Var1" doesn't relate at all with the output, while "Var2" and "Var3" are indeed the most relevant features). However, the contribution scores still aren't properly calculated, as they don't add up to the actual output values.

In [ ]:
explainer = shap.DeepExplainer(model, features[:, :, 2:].float(), feedforward_args=[x_lengths])

In [ ]:
shap_values = explainer.shap_values(features[:, :, 2:].float(), 
                                    feedforward_args=[x_lengths, x_lengths],
                                    var_seq_len=True)

In [ ]:
# Summarize the effects of all the features
shap.summary_plot(shap_values.reshape(-1, model.rnn.input_size), features=features[:, :, 2:].contiguous().view(-1, model.rnn.input_size).numpy(), feature_names=dmy_cols, plot_type='violin')

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
subject = 1

# True sequence length of the current patient's data
seq_len = seq_len_dict[features[subject, 0, 0].item()]

# Plot the explanation of the predictions for one subject
shap.force_plot(explainer.expected_value[0], shap_values[subject, :seq_len], features=features[subject, :seq_len, 2:].numpy(), feature_names=dmy_cols)

In [ ]:
ref_output = model(features[subject, :, 2:].float().unsqueeze(0), [x_lengths[subject]])

In [ ]:
ref_output_s = pd.Series([float(x) for x in list(ref_output.detach().numpy())])

In [ ]:
# Get an overview of the important features and model output for the current patient
dmy_df[dmy_df.subject_id == subject].reset_index().drop(columns='index').assign(output=ref_output_s)

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
ts = 3

# Plot the explanation of the predictions for one subject
shap.force_plot(explainer.expected_value[0], shap_values[subject, ts], features=features[subject, ts, 2:].numpy(), feature_names=dmy_cols)

In my opinion, the problem might be that **SHAP is still letting in padding values**, as if they were realistic values. It seems that it could be due to the use of all the data, in the right side of the multiplication in the definition of the phi variable (contribution score), i.e. when we have something like `phis[l][j] = ... * (x[l] - data[l])`. In my adapted version of the SHAP package, this corresponds to lines 237 and 241.

### Gradient Explainer

In [ ]:
explainer = shap.GradientExplainer(model, features[:, :, 2:].float(), feedforward_args=[x_lengths])

In [ ]:
shap_values = explainer.shap_values(features[:, :, 2:].float(), feedforward_args=[x_lengths], var_seq_len=True, see_progress=True)

In [ ]:
# Summarize the effects of all the features
shap.summary_plot(shap_values.reshape(-1, model.rnn.input_size), features=features[:, :, 2:].contiguous().view(-1, model.rnn.input_size).numpy(), feature_names=dmy_cols, plot_type='violin')

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
subject = 1

# True sequence length of the current patient's data
seq_len = seq_len_dict[features[subject, 0, 0].item()]

# Plot the explanation of the predictions for one subject
shap.force_plot(0.5, shap_values[subject, :seq_len], features=features[subject, :seq_len, 2:].numpy(), feature_names=dmy_cols)

In [ ]:
ref_output = model(features[subject, :, 2:].float().unsqueeze(0), [x_lengths[subject]])

In [ ]:
ref_output_s = pd.Series([float(x) for x in list(ref_output.detach().numpy())])

In [ ]:
# Get an overview of the important features and model output for the current patient
dmy_df[dmy_df.subject_id == subject].reset_index().drop(columns='index').assign(output=ref_output_s)

In [ ]:
# Init the JS visualization code
shap.initjs()

# Choosing which example to use
ts = 3

# Plot the explanation of the predictions for one subject
shap.force_plot(0.5, shap_values[subject, ts], features=features[subject, ts, 2:].numpy(), feature_names=dmy_cols)

The gradient explainer, similarly to the deep explainer, doesn't calculate the contributions correctly, as it should add up to the output value.